In [4]:
#import necessary package and collect initial variables
import openmdao.api as om
%run ../initialvars.ipynb

In [2]:
#define discipline group and call analyses 
class aero(om.Group):
    def setup(self):
        
        self.add_subsystem('d1', analysis1(), promotes_inputs=['d', 'm'], promotes_outputs=['L','W'])
        """
        create analysis subsystem (above) and define default inputs to discipline group (below)
        """
        self.set_input_defaults('d', 1.0) 
        """deflection"""
        self.set_input_defaults('m', 240000) 
        """mass"""   
        self.nonlinear_solver = om.NonlinearBlockGS()

In [3]:
#define analyses, this can be done in different files if wanted
class analysis1(om.ExplicitComponent):
    """
    Sample analysis definition, with inputs & outputs 
    """

    def setup(self):

        """defining input vars"""
        self.add_input('d', val = 0.1)
        self.add_input('m', val = 240000)
    def setup_partials(self):
        """declare whether partials will be used for solving"""
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        Evaluates the equation
        """
        d = inputs['d']
        m = inputs['m']
        
        outputs['L'] = 2*d*(m^2)
        outputs['W'] = 8000*d